In [1]:
import pandas as pd
import numpy as np

In [13]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
date = '2018-04-30'

In [14]:
PTV = pd.read_csv('IPTV_'+str(date)+'_TF1.csv')[['TITRE','DUREE','description programme','HEURE',
       'debut']]
PTV['fin'] = PTV['debut']+PTV['DUREE']
Points = pd.read_csv('merged_'+str(date)+'.csv')[['minutes','heure','label','debut','fin','TITRE','DUREE','description programme', 'temps depuis debut', 'temps avant fin',
       'pourcentage déjà vu']]

Points.head(20)

In [15]:
PTV

,TITRE,DUREE,description programme,HEURE,debut,fin
0,TFou,120,Magazine jeunesse,06:25:00,385,505
1,Météo,5,Météo,08:25:00,505,510
2,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555
3,Météo,5,Météo,09:15:00,555,560
4,Petits secrets entre voisins,30,Série réaliste,09:20:00,560,590
5,Petits secrets entre voisins,35,Série réaliste,09:50:00,590,625
6,Demain nous appartient,35,Série dramatique,10:25:00,625,660
7,Les feux de l'amour,60,Feuilleton sentimental,11:00:00,660,720
8,Les douze coups de midi,55,Jeu,12:00:00,720,775
9,Petits plats en équilibre,5,Magazine de la gastronomie,12:55:00,775,780


C

In [16]:
C = pd.DataFrame(PTV.iloc[[(PTV.shape[0]-1)]]).reset_index(drop=True)
C['Heure réelle'] = 0
C['minute réelle'] = 0
C['retard'] = 0
C['commentaire'] = 'début du programme'
P = PTV.shape[0]-1 #### ligne of programme we are looking at
lastend = 130
lastduree =255
current_prog = C.iloc[0]
start = True
Oups = False
for i in range(Points.shape[0]):
    if(lastend < 120):
        lastend = lastend +1440
    #take point one by one
    falsend = False
    point  = Points.iloc[i]
    verification = True
    
    while(verification):
        #Détection d'un problème dans la notation (oubli d'une fin de programme)
        if((current_prog['debut']+3+lastduree-point['minutes'])<-(min(30,max(0.5*lastduree,5)))):
            
            print("Problème: Fin de "+str(current_prog['TITRE'])+" non détectée") 
            #on écrit une ligne pour prévenir
            C = C.append(current_prog)
            C['commentaire'].iloc[(C.shape[0]-1)] = 'fin de programme non détectée'
            C['Heure réelle'].iloc[(C.shape[0]-1)] = point['heure']
            C['minute réelle'].iloc[(C.shape[0]-1)] = point['minutes']
            #Dans ce cas on Update le programme de référence
            
            if(P==(PTV.shape[0]-1) and start==True):
                P=0
                start = False
            else:
                P=P+1
            if(P>=PTV.shape[0] and start==False):
                print("Plus de programme")
                Oups = True
                break
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],0,'début du programme non détectée']
            current_prog = C.iloc[(C.shape[0]-1)]            
            print("Problème: début de "+str(current_prog['TITRE'])+" non détectée") 
            lastend = PTV['debut'].iloc[P]
            lastduree = PTV['DUREE'].iloc[P]
            
        else:
            verification = False
            
    if(Oups):
        break
    # if False Change Point just pass to the next one
    if (int(point['label']) == 0):
        print(str(PTV['TITRE'].iloc[P])+" faux change point")
    
    # if Pub add a ligne of pub in result
    if (int(point['label']) == 1):
        C.loc[C.shape[0]] = ['PUB',6,'publicité',point['heure'],point['minutes'],0,0,0,0,"début de publicité"]
        print(str(PTV['TITRE'].iloc[P])+" publicité")
        
    
    if (int(point['label']) == 2):
        allgood = True
        # Détection de plusiseurs points correspondant a la même fin d'un unique programme
        if(abs(lastend-point['minutes'])<3):
            print(str(PTV['TITRE'].iloc[P-1])+" detection multiple de fin de programme")
            continue
    
        #Détection de daux points (mauvaise classification)
        elif((lastend+lastduree-point['minutes'])>lastduree*0.5):
            print(str(PTV['TITRE'].iloc[P])+" detection érronée de fin de programme")
            continue
        
        else:
            verification = True
            while(verification):
                #Détection d'un problème dans la notation (oubli d'une fin de programme)
                if((current_prog['debut']+3+lastduree-point['minutes'])<-(min(30,max(0.5*lastduree,5)))):
                    print("PROBLEME") 
                    #on écrit une ligne pour prévenir
                    C = C.append(current_prog)
                    C['commentaire'].iloc[(C.shape[0]-1)] = 'fin de programme(loop) non détectée'
                    #Dans ce cas on Update le programme de référence
                    C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],0,'début du programme non détectée']
                    current_prog = C.iloc[(C.shape[0]-1)]
                    lastend = PTV['debut'].iloc[P]
                    lastduree = PTV['DUREE'].iloc[P]
                # Détection de plusiseurs points correspondant a la même fin d'un unique programme
                elif(abs(lastend-point['minutes'])<3):
                    print(str(PTV['TITRE'].iloc[P-1])+" detection(loop) multiple de fin de programme")
                    allgood = False
                    break

                #Détection de faux points (mauvaise classification)
                elif((lastend+lastduree-point['minutes'])>lastduree*0.5):
                    print(str(PTV['TITRE'].iloc[P])+" detection(loop) érronée de fin de programme")
                    allgood = False
                    break
                else:
                    verification = False
        #si tout ca se passe bien on update
        if(allgood):
            lastend = point['minutes']
            if(P>=PTV.shape[0] and start==False):
                print("Plus de programme")
                break
            
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],min(abs(point['minutes']-PTV['fin'].iloc[P]),abs(point['minutes']%1440-PTV['fin'].iloc[P])),"fin détectée"]
            if(P==(PTV.shape[0]-1) and start==True):
                P=0
                start = False
            else:
                P=P+1
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],C['retard'].iloc[(C.shape[0]-1)],'début de programme']
            current_prog = C.iloc[(C.shape[0]-1)]
            lastduree = PTV['DUREE'].iloc[P]
            C['commentaire'].iloc[(C.shape[0]-1)] = 'début de programme'
            print(str(PTV['TITRE'].iloc[P-1])+" detection de fin de programme")
        else:
            print("j'ai raté un truc")
#display_side_by_side(C,PTV)

/home/alexis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Programmes de la nuit detection de fin de programme
Programmes de la nuit detection multiple de fin de programme
TFou publicité
TFou publicité
TFou faux change point
TFou publicité
TFou faux change point
TFou publicité
TFou publicité
TFou faux change point
TFou detection de fin de programme
Météo faux change point
Problème: Fin de Météo non détectée
Problème: début de Téléshopping non détectée
Téléshopping faux change point
Téléshopping faux change point
Téléshopping faux change point
Téléshopping publicité
Téléshopping faux change point
Téléshopping faux change point
Téléshopping faux change point
Téléshopping detection de fin de programme
Téléshopping detection multiple de fin de programme
Problème: Fin de Météo non détectée
Problème: début de Petits secrets entre voisins non détectée
Petits secrets entre voisins publicité
Petits secrets entre voisins publicité
Problème: Fin de Petits secrets entre voisins non détectée
Problème: début de Petits secrets entre voisins non détectée
Peti

In [17]:
C

,TITRE,DUREE,description programme,HEURE,debut,fin,Heure réelle,minute réelle,retard,commentaire
0,Programmes de la nuit,250,Programme indéterminé,02:15:00,135,385,0,0,0,début du programme
1,Programmes de la nuit,250,Programme indéterminé,02:15:00,135,385,6h29,389,4,fin détectée
2,TFou,120,Magazine jeunesse,06:25:00,385,505,6h29,389,4,début de programme
3,PUB,6,publicité,7h11,431,0,0,0,0,début de publicité
4,PUB,6,publicité,7h23,443,0,0,0,0,début de publicité
5,PUB,6,publicité,7h35,455,0,0,0,0,début de publicité
6,PUB,6,publicité,8h4,484,0,0,0,0,début de publicité
7,PUB,6,publicité,8h22,502,0,0,0,0,début de publicité
8,TFou,120,Magazine jeunesse,06:25:00,385,505,8h28,508,3,fin détectée
9,Météo,5,Météo,08:25:00,505,510,8h28,508,3,début de programme


In [18]:
C.to_html('newPTV.html')

In [19]:
PTV.to_html('PTV.html')

In [20]:
newPTV = C[C["description programme"] != 'publicité']

In [21]:
j = 0
for index, row in newPTV.iterrows():
        modified = False
        while(not modified):
            if(row['commentaire'] == 'début de programme'):
                print(row['TITRE'])
                if(PTV['TITRE'][j] == row['TITRE']):
                    PTV.set_value(j,'debut', row['minute réelle'])
                    modified = True
                    j = j+1
                else:
                    j = j+1
            else:
                modified = True

TFou
Météo
Météo
Météo
Demain nous appartient
Demain nous appartient
Demain nous appartient
Les feux de l'amour
Les douze coups de midi
Petits plats en équilibre
Journal
Petits plats en équilibre
Météo
Warren Jeffs : le gourou polygame
Warren Jeffs : le gourou polygame
Les plus belles mariées
Bienvenue à l'hôtel
Demain nous appartient
Météo
Le 20h le mag
Le 20h le mag
Tirage du Loto
Camping Paradis
Camping Paradis
New York Unité Spéciale
New York Unité Spéciale


/home/alexis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [22]:
PTV.to_csv('PTV_'+str(date)+'_TF1.csv',index=False)

In [23]:
print('done')

done
